In [1]:
#toy_data = [("date_t-1", "result_t-1":"[keyword_t-2]")]
toy_data = [("1", "up" , ["看好","榮景","利多","成長","的"])
        ,("2", "up" , ["看好","榮景","的"])
        ,("3", "up" , ["看好","利多","榮景","看空","的"])
        ,("4", "down" , ["利多","看空","衰退","的"])
        ,("5", "down" , ["看空","衰退","的"])
        ,("6", "un" , ["衰退","利多","成長","的"])
        ,("7", "un" , ["未定","利空","不變","的"])
        ,("8", "un" , ["未定","利空","不變","的"])
        ,("9", "up" , ["看好","利多","見底","看空","的"])
        ,("10", "down" , ["不變","衰退","的"])
        ,("11", "up" , ["看好","大漲","見底","看空","的"])
        ,("12", "down" , ["大跌","衰退","的"])
        ,("13", "up" , ["看好"])
        ,("14", "up" , ["看好"])
           ]

In [2]:
import numpy as np
from scipy.stats import chi2_contingency

#get all stat
last_index = 0
STAT_INDEX_DICT = {}
for data in toy_data:
    if data[1] not in STAT_INDEX_DICT:
        STAT_INDEX_DICT[data[1]] = last_index
        last_index += 1
print(STAT_INDEX_DICT)
#get all phrase
keyword_set = set()
for data in toy_data:
    for keyword_t_2 in data[2]:
        keyword_set.add(keyword_t_2)
print("關鍵字列表：" + str(keyword_set))

#initialize RC table for each keyword
keyword_rc_dict = {}
for keyword in keyword_set:
    keyword_rc_dict[keyword] = np.ones((2, len(STAT_INDEX_DICT)))#np.array([[1, 1, 1], [1, 1, 1]])

#count for each keyword in each RC table
for keyword in keyword_set:
    for data in toy_data:
        result_t = data[1]
        EXIST_INDEX = 0 if keyword in data[2] else 1
        keyword_rc_dict[keyword][EXIST_INDEX][STAT_INDEX_DICT[result_t]] += 1

#run chi-square test for each keyword
#H0:variables are independent(無關) , H1:variables are dependent(有關)
effect_keyword_list = []
for keyword in keyword_rc_dict:
    obs = keyword_rc_dict[keyword]
    (chi2, p, dof, ex) = chi2_contingency(obs, True)
    if p <=0.05:
        #reject H0
        #have correlation
        print("\"" + keyword + "\" 關鍵字與股價變動有關")
        print(p)
        effect_keyword_list.append(keyword)


{'up': 0, 'down': 1, 'un': 2}
關鍵字列表：{'衰退', '利多', '不變', '大漲', '利空', '見底', '的', '看空', '看好', '未定', '成長', '大跌', '榮景'}
"衰退" 關鍵字與股價變動有關
0.019999719369791027
"看好" 關鍵字與股價變動有關
0.007044165766941701


In [3]:

t_1_keyword_candidate_list = ["看好", "榮景", "衰退", "成長"]
today_keyword_list = list(today_keyword for today_keyword in t_1_keyword_candidate_list if today_keyword in effect_keyword_list)

In [4]:
import matplotlib.pyplot as plt  
from wordcloud import WordCloud  
import random

def create_word_cloud(text_to_cloud):
    wc = WordCloud(background_color = "black", 
                   max_words = 2000,
                   font_path = "PingFang.ttc",  
                   max_font_size = 50,
                   random_state = 30,
        )  
    my_word_cloud = wc.generate(text_to_cloud)

    plt.imshow(my_word_cloud)  
    plt.axis("off")  
    plt.show()  

word_count_list = []
if len(today_keyword_list) >= 10:
    for today_keyword in today_keyword_list:
        for _ in range(random.randint(10,20)):
            word_count_list.append(today_keyword)
    word_count_str = " ".join(word_count_list)
    create_word_cloud(word_count_str)
else:
    print("今日關鍵詞：" + ", ".join(today_keyword_list))

今日關鍵詞：看好, 衰退
